In [25]:
# refer to repo .README for necessary format for keys.py



In [2]:
import Cython
from Cython.Compiler import Options

Cython.Compiler.Options.get_directive_defaults()['linetrace'] = True
Cython.Compiler.Options.get_directive_defaults()['binding'] = True

%reload_ext Cython

In [36]:
x = np.array([])

np.append(x, 5)

array([5.])

In [37]:
x

array([], dtype=float64)

In [47]:
%%cython
cimport numpy as np
import pandas as pd

from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('vader_lexicon')

from datetime import timedelta
import datetime
import pytz

from keys import *

cdef client = tweepy.Client(bearer_token=bearer_token,\
                       consumer_key = consumer_key,\
                       consumer_secret = consumer_secret,\
                       access_token = access_token,\
                       access_token_secret = access_token_secret)

cdef auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

cdef df_data = pd.read_csv("nbaData").drop('Unnamed: 0', axis=1)
cdef df_odds = pd.read_csv("nbaOdds.csv")
cdef df_results = pd.read_csv("nbaResults.csv")

df_data.dateTime = pd.to_datetime(df_data.dateTime)

utc=pytz.UTC

cdef weekPriorDate = datetime.datetime.now() - timedelta(days=6)
weekPriorDate = utc.localize(weekPriorDate) 

df_data.dateTime = df_data.dateTime.apply(lambda x: x.replace(tzinfo=None))

cdef study_games = df_data[df_data.dateTime > weekPriorDate.replace(tzinfo=None)].__id__.unique()

def percentage(double part, double whole):
    cdef double prod = 100 * float(part)/float(whole)
    return prod 

def clean_tweets(tweet_list):
    tweet_list = [re.sub('RT @\w+: '," ",x) for x in tweet_list]
    tweet_list = [re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x) for x in tweet_list]
    tweet_list = [x.lower() for x in tweet_list]
    return tweet_list


cdef sentiment_results = pd.DataFrame(columns=['gameID', 'homeTeam', 'awayTeam','homePositive', 'homeNegative', 'homeNeutral', 'awayPositive', 'awayNegative', 'awayNeutral', 'total_tweets_analyzed'])
cdef int search_limit = 500

import time

start_time = time.time()

for game_id in study_games:
    df = df_data[df_data.__id__ == game_id]
    home_team = df['homeTeam'].values[0]
    away_team = df['awayTeam'].values[0]

    game_datetime = pd.to_datetime(df['dateTime'].values[0])
    game_datetime_end = game_datetime
    game_datetime_start = game_datetime - timedelta(hours=5, minutes=0)
    
    
    print (game_id, home_team, away_team)
    
    # home tweets 
    home_tweet_list = []
    home_negative = 0
    home_positive = 0
    home_neutral = 0
    
    for tweet in tweepy.Paginator(client.search_recent_tweets, query=home_team,
                              tweet_fields=['lang', 'created_at'], max_results=100, start_time=game_datetime_start, end_time=game_datetime_end).flatten(limit=search_limit):

        if tweet.lang == 'en':
            home_tweet_list.append(tweet.text)
     
    home_tweet_count = len(set(home_tweet_list))
    
    for tweet in clean_tweets(list(set(home_tweet_list))):
        score = SentimentIntensityAnalyzer().polarity_scores(tweet)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']

        if neg > pos:
            home_negative += 1
        elif pos > neg:
            home_positive += 1
        elif pos == neg:
            home_neutral += 1
    
    # away tweets
    away_tweet_list = []
    away_negative = 0
    away_positive = 0
    away_neutral = 0
    
    for tweet in tweepy.Paginator(client.search_recent_tweets, query=away_team,
                              tweet_fields=['lang', 'created_at'], max_results=100, start_time=game_datetime_start, end_time=game_datetime_end).flatten(limit=search_limit):

        if tweet.lang == 'en':
            away_tweet_list.append(tweet.text)
    
    away_tweet_count = len(set(away_tweet_list))
    
    for tweet in clean_tweets(list(set(away_tweet_list))):
        score = SentimentIntensityAnalyzer().polarity_scores(tweet)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']

        if neg > pos:
            away_negative += 1
        elif pos > neg:
            away_positive += 1
        elif pos == neg:
            away_neutral += 1
    
    sentiment_results = sentiment_results.append(pd.DataFrame([[game_id, home_team, away_team, percentage(home_positive, home_tweet_count), percentage(home_negative, home_tweet_count), percentage(home_neutral, home_tweet_count),
                                        percentage(away_positive, away_tweet_count), percentage(away_negative, away_tweet_count), percentage(away_neutral, away_tweet_count), away_tweet_count + home_tweet_count]], 
                                           columns=['gameID', 'homeTeam', 'awayTeam','homePositive', 'homeNegative', 'homeNeutral', 'awayPositive', 'awayNegative', 'awayNeutral', 'total_tweets_analyzed']))

end_time = time.time()
    
print(end_time - start_time, 'seconds')


Error compiling Cython file:
------------------------------------------------------------
...

start_time = time.time()

for game_id in study_games:
    df = df_data[df_data.__id__ == game_id]
    cdef home_team = df['homeTeam'].values[0]
        ^
------------------------------------------------------------

C:\Users\Nasser\.ipython\cython\_cython_magic_220c13a3d2bdbd022740c29f22542353.pyx:76:9: cdef statement not allowed here


In [29]:
def percentage(part,whole):
    return 100 * float(part)/float(whole)

def clean_tweets(tweet_list):
    tweet_list = [re.sub('RT @\w+: '," ",x) for x in tweet_list]
    tweet_list = [re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x) for x in tweet_list]
    tweet_list = [x.lower() for x in tweet_list]
    return tweet_list

In [20]:
import time

In [31]:
## baseline

start_time = time.time()

sentiment_results = pd.DataFrame(columns=['gameID', 'homeTeam', 'awayTeam','homePositive', 'homeNegative', 'homeNeutral', 'awayPositive', 'awayNegative', 'awayNeutral', 'total_tweets_analyzed'])
search_limit = 500

for game_id in study_games:
    df = df_data[df_data.__id__ == game_id]
    home_team = df['homeTeam'].values[0]
    away_team = df['awayTeam'].values[0]

    game_datetime = pd.to_datetime(df['dateTime'].values[0])
    game_datetime_end = game_datetime
    game_datetime_start = game_datetime - timedelta(hours=5, minutes=0)
    
    
    print (game_id, home_team, away_team)
    
    # home tweets 
    home_tweet_list = []
    home_negative = 0
    home_positive = 0
    home_neutral = 0
    
    for tweet in tweepy.Paginator(client.search_recent_tweets, query=home_team,
                              tweet_fields=['lang', 'created_at'], max_results=100, start_time=game_datetime_start, end_time=game_datetime_end).flatten(limit=search_limit):

        if tweet.lang == 'en':
            home_tweet_list.append(tweet.text)
     
    home_tweet_count = len(set(home_tweet_list))
    
    for tweet in clean_tweets(list(set(home_tweet_list))):
        score = SentimentIntensityAnalyzer().polarity_scores(tweet)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']

        if neg > pos:
            home_negative += 1
        elif pos > neg:
            home_positive += 1
        elif pos == neg:
            home_neutral += 1
    
    # away tweets
    away_tweet_list = []
    away_negative = 0
    away_positive = 0
    away_neutral = 0
    
    for tweet in tweepy.Paginator(client.search_recent_tweets, query=away_team,
                              tweet_fields=['lang', 'created_at'], max_results=100, start_time=game_datetime_start, end_time=game_datetime_end).flatten(limit=search_limit):

        if tweet.lang == 'en':
            away_tweet_list.append(tweet.text)
    
    away_tweet_count = len(set(away_tweet_list))
    
    for tweet in clean_tweets(list(set(away_tweet_list))):
        score = SentimentIntensityAnalyzer().polarity_scores(tweet)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']

        if neg > pos:
            away_negative += 1
        elif pos > neg:
            away_positive += 1
        elif pos == neg:
            away_neutral += 1
    
    sentiment_results = sentiment_results.append(pd.DataFrame([[game_id, home_team, away_team, percentage(home_positive, home_tweet_count), percentage(home_negative, home_tweet_count), percentage(home_neutral, home_tweet_count),
                                        percentage(away_positive, away_tweet_count), percentage(away_negative, away_tweet_count), percentage(away_neutral, away_tweet_count), away_tweet_count + home_tweet_count]], 
                                           columns=['gameID', 'homeTeam', 'awayTeam','homePositive', 'homeNegative', 'homeNeutral', 'awayPositive', 'awayNegative', 'awayNeutral', 'total_tweets_analyzed']))

end_time = time.time()

print(end_time - start_time, 'seconds')

MINMEM4292022 Minnesota Timberwolves Memphis Grizzlies
7.141486883163452 seconds


In [19]:
sentiment_results

,gameID,homeTeam,awayTeam,homePositive,homeNegative,homeNeutral,awayPositive,awayNegative,awayNeutral,total_tweets_analyzed
0,MINMEM4292022,Minnesota Timberwolves,Memphis Grizzlies,42.23301,11.165049,46.601942,45.454545,7.509881,47.035573,459


In [144]:
sentiment_results.to_csv('sentiment_results.csv', index=False)